In [1]:
# First, vectorize the CSV data
import csv
import numpy as np

fname = "creditcard.csv"

all_features = []
all_targets = []
with open(fname) as f:
    for i, line in enumerate(f):
        if i == 0:
            print("HEADER:", line.strip())
            continue  # Skip header
        fields = line.strip().split(",")
        all_features.append([float(v.replace('"', "")) for v in fields[:-1]])
        all_targets.append([int(fields[-1].replace('"', ""))])
        if i == 1:
            print("EXAMPLE FEATURES:", all_features[-1])

features = np.array(all_features, dtype="float32")
targets = np.array(all_targets, dtype="uint8")
print("features.shape:", features.shape)
print("targets.shape:", targets.shape)

HEADER: "Time","V1","V2","V3","V4","V5","V6","V7","V8","V9","V10","V11","V12","V13","V14","V15","V16","V17","V18","V19","V20","V21","V22","V23","V24","V25","V26","V27","V28","Amount","Class"
EXAMPLE FEATURES: [0.0, -1.3598071336738, -0.0727811733098497, 2.53634673796914, 1.37815522427443, -0.338320769942518, 0.462387777762292, 0.239598554061257, 0.0986979012610507, 0.363786969611213, 0.0907941719789316, -0.551599533260813, -0.617800855762348, -0.991389847235408, -0.311169353699879, 1.46817697209427, -0.470400525259478, 0.207971241929242, 0.0257905801985591, 0.403992960255733, 0.251412098239705, -0.018306777944153, 0.277837575558899, -0.110473910188767, 0.0669280749146731, 0.128539358273528, -0.189114843888824, 0.133558376740387, -0.0210530534538215, 149.62]
features.shape: (284807, 30)
targets.shape: (284807, 1)


In [2]:
# Prepare a validation set
num_val_samples = int(len(features) * 0.2)
train_features = features[:-num_val_samples]
train_targets = targets[:-num_val_samples]
val_features = features[-num_val_samples:]
val_targets = targets[-num_val_samples:]

print("Number of training samples:", len(train_features))
print("Number of validation samples:", len(val_features))

Number of training samples: 227846
Number of validation samples: 56961


In [3]:
# Analyze class imbalance in the targets
counts = np.bincount(train_targets[:, 0])
print(
    "Number of positive samples in training data: {} ({:.2f}% of total)".format(
        counts[1], 100 * float(counts[1]) / len(train_targets)
    )
)

weight_for_0 = 1.0 / counts[0]
weight_for_1 = 1.0 / counts[1]

Number of positive samples in training data: 417 (0.18% of total)


In [4]:
# Normalize the data using training set statistics
mean = np.mean(train_features, axis=0)
train_features -= mean
val_features -= mean
std = np.std(train_features, axis=0)
train_features /= std
val_features /= std


In [5]:
# Build a binary classification model
from tensorflow import keras

model = keras.Sequential(
    [
        keras.layers.Dense(
            256, activation="relu", input_shape=(train_features.shape[-1],)
        ),
        keras.layers.Dense(256, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(256, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(1, activation="sigmoid"),
    ]
)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               7936      
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 139,777
Trainable params: 139,777
Non-trainable params: 0
__________________________________________________

In [6]:
# Train the model with class_weight argument
metrics = [
    keras.metrics.FalseNegatives(name="fn"),
    keras.metrics.FalsePositives(name="fp"),
    keras.metrics.TrueNegatives(name="tn"),
    keras.metrics.TruePositives(name="tp"),
    keras.metrics.Precision(name="precision"),
    keras.metrics.Recall(name="recall"),
]

model.compile(
    optimizer=keras.optimizers.Adam(1e-2), loss="binary_crossentropy", metrics=metrics
)

callbacks = [keras.callbacks.ModelCheckpoint("fraud_model_at_epoch_{epoch}.h5")]
class_weight = {0: weight_for_0, 1: weight_for_1}

model.fit(
    train_features,
    train_targets,
    batch_size=2048,
    epochs=30,
    verbose=2,
    callbacks=callbacks,
    validation_data=(val_features, val_targets),
    class_weight=class_weight,
)

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Epoch 1/30
112/112 - 8s - loss: 2.1591e-06 - fn: 41.0000 - fp: 19387.0000 - tn: 208042.0000 - tp: 376.0000 - precision: 0.0190 - recall: 0.9017 - val_loss: 0.1734 - val_fn: 6.0000 - val_fp: 2485.0000 - val_tn: 54401.0000 - val_tp: 69.0000 - val_precision: 0.0270 - val_recall: 0.9200
Epoch 2/30
112/112 - 5s - loss: 1.5606e-06 - fn: 31.0000 - fp: 8198.0000 - tn: 219231.0000 - tp: 386.0000 - precision: 0.0450 - recall: 0.9257 - val_loss: 0.0270 - val_fn: 13.0000 - val_fp: 68.0000 - val_tn: 56818.0000 - val_tp: 62.0000 - val_precision: 0.4769 - val_recall: 0.8267
Epoch 3/30
112/112 - 4s - loss: 1.2809e-06 - fn: 32.0000 - fp: 7469.0000 - tn: 219960.0000 - tp: 385.0000 - precision: 0.0490 - recall: 0.9233 - val_loss: 0.0799 - val_fn: 8.0000 - val_fp: 1745.0000 - val_tn: 55141.0000 - val_tp: 67.0000 - val_precision: 0.0370 - val_recall: 0.8933
Epoch 4/30
11

Epoch 29/30
112/112 - 4s - loss: 3.3417e-07 - fn: 3.0000 - fp: 2429.0000 - tn: 225000.0000 - tp: 414.0000 - precision: 0.1456 - recall: 0.9928 - val_loss: 0.0306 - val_fn: 9.0000 - val_fp: 596.0000 - val_tn: 56290.0000 - val_tp: 66.0000 - val_precision: 0.0997 - val_recall: 0.8800
Epoch 30/30
112/112 - 4s - loss: 3.7633e-07 - fn: 4.0000 - fp: 3377.0000 - tn: 224052.0000 - tp: 413.0000 - precision: 0.1090 - recall: 0.9904 - val_loss: 0.0235 - val_fn: 8.0000 - val_fp: 528.0000 - val_tn: 56358.0000 - val_tp: 67.0000 - val_precision: 0.1126 - val_recall: 0.8933
